In [ ]:
from PIL import Image
import os
from patchify import patchify
import tifffile as tiff
import numpy as np

In [ ]:
#przetwarza zdjęcia na czarnobiałe, na takich był uczony model
DATA_PATH = "your_path_to_aerial_photos"
limit = 0

for i in os.listdir(DATA_PATH):
    for x in os.listdir(DATA_PATH+i+"/"):
        img = Image.open(DATA_PATH+i+"/"+x).convert("RGBA")
        bgimg = Image.new("RGBA", img.size, "WHITE")
        fix= Image.alpha_composite(bgimg, img)
        fix.save(DATA_PATH+i+"/"+x)
        img = Image.open(DATA_PATH+i+"/"+x)
        img = img.convert("L")
        img.save(DATA_PATH+i+"/"+x)
        img = Image.open(DATA_PATH+i+"/"+x)
        img = img.convert("RGB")
        img.save(DATA_PATH+i+"/"+x)
        print(x)
    limit +=1
    print(limit)

In [ ]:
#znajduje najbliższy rozmiar obrazu większy od obrazu zdjęcia lotniczego lub jemu równy,
#a następnie tworzy białe tło i nakłada na nie zdjęcie lotnicze
def image_512div(img, path):
    if ((img.size[0])%512 != 0) or ((img.size[1])%512 != 0):
        size_x = (int(img.size[0] / 512) + (img.size[0] % 512 > 0))*512
        size_y = (int(img.size[1] / 512) + (img.size[1] % 512 > 0))*512
        print(str(size_x)+", "+str(size_y))
        bgimg = Image.new("RGBA", [size_x,size_y], "WHITE")
        bgimg.paste(img,[0,0])
        bgimg.save(path)

In [ ]:
#tworzy fragmenty zdjecia lotniczego rozmiarow 512 na 512 pikseli i zapisuje je w istniejacych folderach i tworzy
#foldery do zapisu jesli ich nie ma
#istnieje przy zalozeniu struktury sciezki do plikow dysk/.../folder z folderami/folder ze zdjeciami konkretnego jeziora
DATA_PATH = "your_path_to_aerial_photos"
limit = 0
for i in os.listdir(DATA_PATH):
    for x in os.listdir(DATA_PATH+i+"/"):
        img = Image.open(DATA_PATH+i+"/"+x).convert("RGB")
        image_512div(img,(DATA_PATH+i+"/"+x))
        large_img = Image.open(DATA_PATH+i+"/"+x).convert("RGB")
        large_img = np.array(large_img)
        patches_img = patchify(large_img, (512,512,3), step=512)
        patches_img=np.squeeze(patches_img)
        if(len(patches_img.shape)>3):
            for j in range(patches_img.shape[0]):
                for k in range(patches_img.shape[1]):
                    single_patch = patches_img[j,k,:]
                    directory = ("your_path_to_output_folder_for_patched_images"+i)
                    if not os.path.exists(directory):
                        os.makedirs(directory)
                    tiff.imwrite(directory+"/"+"_"+str(j)+"_"+str(k)+x, single_patch)
        else:
            directory = ("your_path_to_output_folder_for_patched_images/"+i)
            if not os.path.exists(directory):
                        os.makedirs(directory)
            tiff.imwrite(directory+"/"+"_"+str(j)+"_"+str(k)+x, large_img)
    limit +=1
    print("folder: "+str(limit))
    #if limit == "insert your number of folders you want to go through here":
    #    break